In [14]:
from scipy.stats import mode
from numpy import linalg as LA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numba
import time
from scipy.sparse.linalg import eigs
from numpy.linalg import inv
import sys
from PFSA import PFSA

<hr style="border:2px solid blue"> </hr>

## load the in-core and ex-core detector signal

In [15]:
# import the in-core and ex-core detector signal
Xn_IC = np.load("X_incore_normal.npz")
Xa_IC = np.load("X_incore_abnormal.npz")

print(Xn_IC.shape)
print(Xa_IC.shape)

(1000000001,)
(1000000001,)


<hr style="border:2px solid blue"> </hr>

## downsampling to fs = 100 Hz

In [16]:
Xn_IC =  Xn_IC[0:-1:10]
Xa_IC =  Xa_IC[0:-1:10]
print(Xn_IC.shape)
print(Xa_IC.shape)

(100000000,)
(100000000,)


<hr style="border:2px solid blue"> </hr>

## investigate the effect of window size on the accuracy of the PFSA for neutron noise anomaly detection

In [17]:
w_sizes = [10, 20, 40, 80, 100, 200, 400, 800, 1000] # window sizes, seconds

### part 1: for  in-core signal

In [18]:
acc = []
for window_i in w_sizes:
    
    X_class1 = Xn_IC[:] # class 1 is normal, index is 0
    X_class2 = Xa_IC[:] # class 2 is abnormal, index is 1
    
    fs = 100
    Ns = 100000000
    
    window = int(fs * window_i)
    sample_size = len(X_class1) // window
    X_class1 = X_class1.reshape(sample_size, window)
    X_class1 = X_class1.reshape(*X_class1.shape, 1)
    np.random.shuffle(X_class1)
    Y_class1 = np.zeros(sample_size) # class 1 is labeled by index 0

    X_class2 = X_class2.reshape(sample_size, window)
    X_class2 = X_class2.reshape(*X_class2.shape, 1)
    np.random.shuffle(X_class2)
    Y_class2 = np.ones(sample_size) # class 2 is labled by index 1

    X_train = np.zeros((X_class1.shape[0], X_class1.shape[1], X_class1.shape[2]))
    Y_train = np.zeros(Y_class1.size)

    X_train[0: sample_size // 2] = X_class1[0: sample_size // 2].copy()
    X_train[sample_size // 2 : ] = X_class2[0: sample_size // 2].copy()

    Y_train[0: sample_size // 2] = Y_class1[0: sample_size // 2].copy()
    Y_train[sample_size // 2 : ] = Y_class2[0: sample_size // 2].copy()

    X_test = np.zeros((X_class1.shape[0], X_class1.shape[1], X_class1.shape[2]))
    Y_test = np.zeros(Y_class1.size)

    X_test[0: sample_size // 2] = X_class1[sample_size // 2:].copy()
    X_test[sample_size // 2 : ] = X_class2[sample_size // 2:].copy()

    Y_test[0: sample_size // 2] = Y_class1[sample_size // 2:].copy()
    Y_test[sample_size // 2 : ] = Y_class2[sample_size // 2:].copy()
    
    classfiers = ['projection 2'] # select to only use projection 2 classifier

    n_partitioning = 50
    depth = 1

    print("fs:", fs, "Ns:", Ns, "window:", window, "n_partitioning", n_partitioning)
    normalizaiton_method = 'mixed classes' # select to use "mixed classes" normalization method
    model = PFSA(n_partitioning = n_partitioning, depth = depth, train_regime= normalizaiton_method)
    model.fit(X_train, Y_train)
    y_test_predicted = model.predict(X_test, classifier=classfiers)
    acc.append(sum(y_test_predicted  == Y_test) / len(Y_test))

fs: 100 Ns: 100000000 window: 1000 n_partitioning 50
normalization for all classes together
get the same bounds for all classes
get symbols for all classes
get states for all classes
get morph matrix 0.000000
get morph matrix 1.000000
class 0.000000, calculate the left eigenvector corresponding to left eigenvalue 1, this is the state probability vector.
class 0.000000, calculate the right eigenvectors corresponding to right eigenvalue (excludes eigen value 1).
class 1.000000, calculate the left eigenvector corresponding to left eigenvalue 1, this is the state probability vector.
class 1.000000, calculate the right eigenvectors corresponding to right eigenvalue (excludes eigen value 1).
class 0.000000, calculate state weight Chi
class 0.000000, calculate projection matrix 1
class 0.000000, calculate projection matrix 2
class 1.000000, calculate state weight Chi
class 1.000000, calculate projection matrix 1
class 1.000000, calculate projection matrix 2
class 0
normalization
get states fo

class 0
normalization
get states for all classes
get morph matrix
class 1
normalization
get states for all classes
get morph matrix
get labels
fs: 100 Ns: 100000000 window: 100000 n_partitioning 50
normalization for all classes together
get the same bounds for all classes
get symbols for all classes
get states for all classes
get morph matrix 0.000000
get morph matrix 1.000000
class 0.000000, calculate the left eigenvector corresponding to left eigenvalue 1, this is the state probability vector.
class 0.000000, calculate the right eigenvectors corresponding to right eigenvalue (excludes eigen value 1).
class 1.000000, calculate the left eigenvector corresponding to left eigenvalue 1, this is the state probability vector.
class 1.000000, calculate the right eigenvectors corresponding to right eigenvalue (excludes eigen value 1).
class 0.000000, calculate state weight Chi
class 0.000000, calculate projection matrix 1
class 0.000000, calculate projection matrix 2
class 1.000000, calculate

In [19]:
print(acc)
np.save("acc_incore_window_sizes_fs100.npy", acc)

[0.64837, 0.70332, 0.77596, 0.85792, 0.8843, 0.956, 0.9912, 1.0, 1.0]
